In [1]:
!pip install mediapipe

In [4]:
!pip install xlsxwriter

In [2]:
import mediapipe as mp
import cv2
#import skvideo.io
#import skvideo.datasets
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [4]:
mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)


DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)

In [5]:
def pos_to_dict(pose_landmarks):
    if pose_landmarks != None:
        keypoints = []
        for data_point in pose_landmarks.landmark:
            keypoints.append({
                                 'X': data_point.x,
                                 'Y': data_point.y,
                                 'Z': data_point.z,
                                 })
        return keypoints


In [6]:
cap = cv2.VideoCapture(r"C:\Users\hagar\OneDrive - mail.tau.ac.il\Desktop\CUSTIME videos\custime_sentences\sent_01.mp4")
cap.set(3,640)
cap.set(4,480)

# We need to set resolutions.
# so, convert them from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)

# Below VideoWriter object will create
# a frame of above defined The output
# is stored in 'filename.avi' file.
result = cv2.VideoWriter('estimated_sentence1.avi',cv2.VideoWriter_fourcc(*'MJPG'),30, size)



# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        
        # Save the landmarks to dfs
        face_lm = pos_to_dict(results.face_landmarks)
        face_d = pd.DataFrame(face_lm)
        right_hand_lm = pos_to_dict(results.right_hand_landmarks)
        right_hand_d = pd.DataFrame(right_hand_lm)
        
        # Make Detections
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(0,0,0), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
         #                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
          #                       mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
           #                      )
        
        cv2.imshow('cued_estimated', image)
        result.write(image)


        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

#writing the landmarks to xlsx
with pd.ExcelWriter('estimate_cordinates.xlsx') as writer:  
    face_d.to_excel(writer, sheet_name='face_lm')
    right_hand_d.to_excel(writer, sheet_name='right_hand_lm')


result.release()
cap.release()
cv2.destroyAllWindows()
print("The video was successfully saved")


The video was successfully saved


NameError: name 'writer' is not defined